Install python-docx

In [1]:
!pip install python-docx
!pip install openpyxl

Load the libraries into the Python Requirement

In [2]:
from docx import Document
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 

In [3]:
# Load the dataset 
import openpyxl
import pandas as pd

wb = openpyxl.load_workbook('SDG Reports 2021_Tracking log.xlsx', data_only=True)
sheet = wb.worksheets[1]

dat = pd.DataFrame(sheet.values)
dat = dat.rename(columns=dat.iloc[0]).drop([0])
df =  dat[dat['Type']=='Storyline'][['Indicator', 'Sender email', 'Sender_name', 'Sender_agency', 'File name']]

# Drop duplicates with the same indicator, sender email, sender name and sender agency
df.drop_duplicates(keep = 'last', inplace = True)

# Keep the row that contains sender_agency: World Bank (the first one) and remove the second one (UNDRR)
df = df.drop(510)
df 

,Indicator,Sender email,Sender_name,Sender_agency,File name
2,1.1.1 Proportion of the population living belo...,kapsos@ilo.org,Steven Kapsos,ILO,None
5,1.1.1 Proportion of the population living belo...,userajuddin@worldbank.org,Umar Serajuddin,World Bank,s_1.1.1_World Bank_2021.04.16_rev.docx
8,1.2.1 Proportion of population living below th...,dmahler@worldbank.org,Daniel Mahler,World Bank,s_1.2.1_World Bank_2021.04.16_rev.docx
11,"1.2.2 Proportion of men, women and children of...",None,None,None,None
14,1.3.1 Proportion of population covered by soci...,kapsos@ilo.org,Steven Kapsos,ILO,s_1.3.1_ILO_2021.04.16_rev.docx
...,...,...,...,...,...
742,17.18.1 Statistical capacity indicator for Sus...,None,None,None,None
745,17.18.2 Number of countries that have national...,Yu.TIAN@oecd.org,Yu Tian,PARIS21,s_17.18.2_PARIS21_2021.03.11.docx
748,17.18.3 Number of countries with a national st...,Yu.TIAN@oecd.org,Yu Tian,PARIS21,s_17.18.3_PARIS21_2021.03.11.docx
751,17.19.1 Dollar value of all resources made ava...,Yu.TIAN@oecd.org,Yu Tian,PARIS21,s_17.19.1_PARIS21_2021.03.11.docx


In [4]:
# Extract the storyline from the word document

def extract_doc(new_doc, file_name, indicator, sender_name, sender_agency, sender_email, not_standard_format_files_list):

    docx_dict = {}
    index = 0
    document = Document(file_name)
    
    words = '2-4 sentences with summary of main findings and supporting data:'
    condition = False
    text = []
    for para in document.paragraphs:
        if words in para.text:
            condition = True
        if condition: 
            text = np.append(text, para.text).tolist()
    while '' in text:
        text.remove('')
    

    if len(text) != 0:
        storyline = text[1]
        # Add the name, sender name, email and agency 
        title = indicator + " | Submitted by: "  + sender_name + ', ' + sender_agency + ' (' + sender_email + ')'
        # Write the context into the document
        new_doc.add_paragraph(title, style = "Heading 3")
        new_doc.add_paragraph(storyline)
    
    else:
        not_standard_format_files_list = np.append(not_standard_format_files_list, file_name)

    return not_standard_format_files_list 

In [5]:
# Get the name of the files, sender emails, sender names, and sender agency
indicators = df['Indicator'].tolist()
file_name = df['File name'].tolist()
sender_names = df['Sender_name'].tolist()
sender_emails = df['Sender email'].tolist()
agencies = df['Sender_agency'].tolist()

In [6]:
# Extract the information and save it to the document
new_doc = Document()
not_standard_format_files_list = []
for filename in os.listdir():
    for name in file_name: 
        if name == filename:
            idx = file_name.index(name)
            sender_name = sender_names[idx]
            email = sender_emails[idx]
            agency = agencies[idx]
            indicator = indicators[idx]
            not_standard_format_files_list = extract_doc(new_doc, filename, indicator, sender_name, agency, email, not_standard_format_files_list)
            
        
# Save the new document
new_doc.save('output.docx')

### Word Documents Not Followed Standard Format

In [7]:
not_standard_format_files_list

array(['s_7.1.2_WHO_2021.03.04.docx'], dtype='<U32')